# 챕터 18: asyncio를 이용한 동시성

* 간단한 스레드 프로그램과 그에 준하는 asyncio 버전을 비교하면서, 스레드와 비동기 작업의 관계를 보여준다.
* asyncio.Future 클래스와 concurrent.futures.Future 클래스의 차이점을 설명한다.
* 17장에서 구현한 국기 내려받기 예제의 비동기 버전을 구현한다.
* 스레드나 프로세스를 사용하지 않고 비동기 프로그래밍이 네트워크 프로그램에서 높은 동시성을 관리하는 방법을 설명한다.
* 코루틴으로 비동기 프로그래밍을 하기 위한 콜백을 개선시키는 방법을 설명한다.
* 블로킹 연산을 스레드 풀에 덜어줌으로써 이벤트 루프를 블로킹하지 않는 방법을 알아본다.
* asyncio 서버를 작성하고, 웹 애플리케이션의 높은 동시성을 다시 생각해본다.
* asyncio 파이썬 생태계에서 커다란 영향을 줄 수밖에 없는 이유를 설명한다.

## 18.1 스레드와 코루틴 비교

In [ ]:
# 예제 18-1: spinner_thread.py: 스레드로 텍스트 스피너 애니메이트하기
import threading
import itertools
import time
import sys

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
def slow_function():
    # 입출력을 위해 장시간 기다리는 것처럼 보이게 만든다.
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.join()
    return result

def main():
    result = supervisor()
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

In [ ]:
# 예제 18-2: spinner_asyncio.py: 코루틴으로 텍스트 스피너 애니메이트하기
import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' +msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
@asyncio.coroutine
def slow_function():
    #입출력을 위해 장시간 기다리는 것처럼 보이게 만든다.
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

In [ ]:
# 예제 18-3: spinner_thread.py: 스레드화된 supervisor() 함수
def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

In [ ]:
# 예제 18-4: spinner_asyncio.py: 비둥기 supervisor() 코루틴
@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

## 18.2 asyncio와 aiohttp로 내려받기

In [ ]:
# flags_asyncio.py: asyncio와 aiohttp를 사용한 비둥기 내려받기 스크립트
import asyncio

import aiohttp

from flags import BASE_URL,save_flag, show, main

@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    image = yield from resp.read()
    return image

@asyncio.coroutine
def download_one(cc):
    image = yield from get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    loop = asyncio.get_event_loop()
    to_do = [download_one(cc) for cc in sorted(cc_list)]
    wait_coro = asyncio.wait(to_do)
    res, _ = loop.run_until_complete(wait_coro)
    loop.close()
    
    return len(res)

if __name__ == '__main__':
    main(download_many)

## 18.3 블로킹 호출을 에둘러 실행하기

## 18.4 asyncio 내려받기 스크립트 개선
## 18.4.1 asyncio.as_completed() 사용하기

In [ ]:
# flags2_asyncio.py: 스크립트의 앞부분
import asyncio
import collections
import aiohttp
from aiohttp import web
import tqdm

from flags2_common import main, HTTPStatus, Result, save_flag

# 원격 서버에서 '503 - Service Temporarily Unavailable'과 같은
# 오류가 생기지 않도록 기본값을 낮게 설정한다.
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000

class FetchError(Exception):
    def __init__(self, country_code):
        self.country_code = country_code

@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    if resp.status == 200:
        image = yield from resp.read()
        return image
    elif resp.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.HttpProcessingError(
            code=resp.status, message=resp.reason,
            headers=resp.headers)
        
@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with(yield from semaphore):
            image = yield from get_flag(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'OK'
        
    if verbose and msg:
        print(cc, msg)
return Results(status, cc)